In [1]:
import tables
tables.file._open_files.close_all()
for i in range(11):
    y=i+1
    print('##############################################################################')
    print('Iteration #{0}'.format(y))
    print('##############################################################################')
    #training
    from numpy.random import seed
    seed(1)
    from tensorflow import set_random_seed
    set_random_seed(2)
    import warnings; warnings.simplefilter('ignore')

    from nilmtk import DataSet
    train = DataSet('redd.h5')
    
    train.set_window(end="30-4-2011") #Use data only until 4/30/2011
    train_elec = train.buildings[1].elec


    from grudisaggregator import GRUDisaggregator
    gru = GRUDisaggregator()
    train_mains = train_elec.mains().all_meters()[0] # The aggregated meter that provides the input
    train_meter = train_elec.submeters()['microwave'] # The microwave meter that is used as a training target

    gru.train(train_mains, train_meter, epochs=4, sample_period=12)
    #gru.export_model("model-redd12b.h5")
    
    #testing
    test = DataSet('redd.h5')
    test.set_window(start="30-4-2011")
    test_elec = test.buildings[1].elec
    test_mains = test_elec.mains().all_meters()[0]
    
    # dis == The filename of the resulting datastore
    x= i+1
    str1 = str(x)
    str2 = '.h5'
    dis  = str1+str2
    
    #dis = '2.h5' 
    
    from nilmtk.datastore import HDFDataStore
    output = HDFDataStore(dis, 'w')

    # test_mains: The aggregated signal meter
    # output: The output datastore
    # train_meter: This is used in order to copy the metadata of the train meter into the datastore
    gru.disaggregate(test_mains, output, test_elec['microwave'], sample_period=12)
    result = DataSet(dis)
    res_elec = result.buildings[1].elec
    predicted = res_elec['microwave']
    ground_truth = test_elec['microwave']

    import matplotlib.pyplot as plt
    predicted.plot()
    ground_truth.plot()
    plt.show()
    import metrics
    rpaf = metrics.recall_precision_accuracy_f1(predicted, ground_truth)
    print("============ Recall: {}".format(rpaf[0]))
    print("============ Precision: {}".format(rpaf[1]))
    print("============ Accuracy: {}".format(rpaf[2]))
    print("============ F1 Score: {}".format(rpaf[3]))

    print("============ Relative error in total energy: {}".format(metrics.relative_error_total_energy(predicted, ground_truth)))
    print("============ Mean absolute error(in Watts): {}".format(metrics.mean_absolute_error(predicted, ground_truth)))
    tables.file._open_files.close_all()

##############################################################################
Iteration #1
##############################################################################


C:\Users\nick_\.conda\envs\nilmtk-env2\lib\site-packages\nilm_metadata\file_management.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  objs = yaml.load(fh)
E:\WorkSpace\Python\Notebooks\thesis\NILM-Stacking-master\NILM-Stacking-master\GRU\grudisaggregator.py:113: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(mainlist) == len(meterlist), "Number of main and meter channels should be equal")
Using TensorFlow backend.
C:\Users\nick_\.conda\envs\nilmtk-env2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\nick_\.conda\envs\nilmtk-env2\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyOb

Epoch 1/4
83589/83589 [==============================] - 24s 290us/step - loss: 9.1566e-04
Epoch 2/4
83589/83589 [==============================] - 17s 199us/step - loss: 7.7148e-04
Epoch 3/4
83589/83589 [==============================] - 16s 197us/step - loss: 7.0903e-04
Epoch 4/4
83589/83589 [==============================] - 17s 200us/step - loss: 6.9750e-04


C:\Users\nick_\.conda\envs\nilmtk-env2\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


New sensible chunk: 10124
New sensible chunk: 9389
New sensible chunk: 7315
New sensible chunk: 4508
New sensible chunk: 223
New sensible chunk: 2794
New sensible chunk: 11546
New sensible chunk: 2710
New sensible chunk: 2273
New sensible chunk: 2904
New sensible chunk: 8404


AttributeError: 'GRUDisaggregator' object has no attribute '_save_metadata_for_disaggregation_custom'